In [1]:
from IPython import display as ipythondisplay
from torch import nn
from tqdm.autonotebook import tqdm
from transformers import AutoTokenizer, AutoModel
import cv2
import gc
import math
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import time
import torch
import torch.nn.functional as F
import yaml
from transformers import AutoTokenizer, AutoModel
from dataloader import make_dataloader,dataset_to_loader
from models.CLIP import CLIPModel, AvgMeter
from models.Graph_Model import GNNEncoder
from models.LSTM2 import LSTMDecoder
from models.GlobalFeatureExtractor import GlobalFeatureExtractor

/scratch/pbs.5152609.kman.restech.unsw.edu.au/ipykernel_270318/1573852760.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
[nltk_data] Downloading package wordnet to /home/z5313504/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Inference Code
def get_image_embeddings(loader, model_path,device):
    # tokenizer = DistilBertTokenizer.from_pretrained(CFG.text_tokenizer)
    #tokenizer = AutoTokenizer.from_pretrained(CFG.clinical_encoder_model)
    config_file_path =  "config/config.yaml"
    with open(config_file_path, "r") as conf_file:
        args = yaml.full_load(conf_file)
    
    # model = CLIPModel(args,device).to(device)
    model = torch.load(model_path, map_location=device)
    model.eval()
    
    valid_image_embeddings = []
    
    total_samples = len(loader.dataset)

    total_step = math.ceil(total_samples / loader.batch_size) 
    with torch.no_grad():
        for _ in range(total_step):
            cg, tg, assign_mat, caption_tokens, labels, caption, images,_ = next(iter(loader))
            cg = cg.to(device)
            tg = tg.to(device)
            images = images.to(device)
            caption_tokens = caption_tokens.to(device) 
            graph_out = model.graph_encoder(cg,tg,assign_mat,images)
            global_feat = model.feature_extractor(images)
            merged_feat = torch.cat((graph_out, global_feat), dim=1)
            image_embeddings = model.image_projection(merged_feat)
            valid_image_embeddings.append(image_embeddings)
    return model, torch.cat(valid_image_embeddings)

def get_text_embeddings(loader,model_path,device):
    config_file_path =  "config/config.yaml"
    with open(config_file_path, "r") as conf_file:
        args = yaml.full_load(conf_file)
    tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
    
    model = CLIPModel(args,device).to(device)
    model = torch.load(model_path, map_location=device)
    model.eval()
    
    total_samples = len(loader.dataset)

    total_step = math.ceil(total_samples / loader.batch_size) 
    valid_text_embeddings = []
    with torch.no_grad():
        for _ in range(total_step):
            cg, tg, assign_mat, caption_tokens, labels, caption, images,attention_masks = next(iter(loader))
            # cg = cg.to(device)
            # tg = tg.to(device)
            # images = images.to(device)
            attention_masks = attention_masks.to(device)
            caption_tokens = caption_tokens.to(device) 
            text_features = model.text_encoder(
                caption_tokens,attention_masks
            )
            text_embeddings = model.text_projection(text_features)
            valid_text_embeddings.append(text_embeddings)
    return model, torch.cat(valid_text_embeddings)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_dl,train_dataset = make_dataloader(
        batch_size = 16,
        split = "train",
        base_data_path = "../../../../../../srv/scratch/bic/peter/Report",
        graph_path = "../../../../../../srv/scratch/bic/peter/full-graph",
        vocab_path = "new_vocab_bladderreport.pkl",
        shuffle=True,
        num_workers=0,
        load_in_ram = True
    )
    # #train_dl = get_sample_samplier(train_dataset,args["batch_size"])
    # print(f"train loader size {len(train_dl)}")

test_dl,_ = make_dataloader(
        batch_size =16, # there are 1000 set 1 because we will calculate pair by pair
        split = "test",
        base_data_path = "../../../../../../srv/scratch/bic/peter/Report",
        graph_path = "../../../../../../srv/scratch/bic/peter/full-graph",
        vocab_path = "new_vocab_bladderreport.pkl",
        shuffle=True,
        num_workers=0,
        load_in_ram = True
    )

eval_dl,_ = make_dataloader(
        batch_size = 16, # there are 889 in eval set
        split = "eval",
        base_data_path = "../../../../../../srv/scratch/bic/peter/Report",
        graph_path = "../../../../../../srv/scratch/bic/peter/full-graph",
        vocab_path = "new_vocab_bladderreport.pkl",
        shuffle=True,
        num_workers=0,
        load_in_ram = True
    )

['../../../../../../srv/scratch/bic/peter/full-graph/cell_graphs/train/115833_000.bin', '../../../../../../srv/scratch/bic/peter/full-graph/cell_graphs/train/115833_001.bin']
length report 2336
['../../../../../../srv/scratch/bic/peter/full-graph/cell_graphs/test/115831_000.bin', '../../../../../../srv/scratch/bic/peter/full-graph/cell_graphs/test/115831_001.bin']
length report 998
['../../../../../../srv/scratch/bic/peter/full-graph/cell_graphs/eval/115831_002.bin', '../../../../../../srv/scratch/bic/peter/full-graph/cell_graphs/eval/115831_003.bin']
length report 889


In [4]:
!ls ../../../../../../srv/scratch/bic/peter/CLIP_save


best_128.pt  best_256.pt  best_64.pt  best.pt


In [5]:
config_file_path =  "config/config.yaml"
with open(config_file_path, "r") as conf_file:
    args = yaml.full_load(conf_file)

# model = CLIPModel(args,device).to(device)
model = torch.load("../../../../../../srv/scratch/bic/peter/CLIP_save/best_128.pt", map_location=device)
for name, param in model.named_parameters():
    #if 'fc' not in name:  # or any other condition based on layer names
    param.requires_grad = True

In [10]:

model, image_embeddings = get_image_embeddings(eval_dl, "../../../../../../srv/scratch/bic/peter/CLIP_save/best_64.pt",device)
print(image_embeddings.shape)

torch.Size([896, 512])


In [12]:
import warnings
warnings.filterwarnings("ignore")
model, text_embeddings = get_text_embeddings(eval_dl,"../../../../../../srv/scratch/bic/peter/CLIP_save/best_64.pt",device)
print(text_embeddings.shape)

torch.Size([896, 512])


In [ ]:
def get_matches():
    pass

In [ ]:
    import os
    print("DATALOADER")
    split = "eval"
    splits = ["train","test","eval"]
    loader,_ = make_dataloader(
        batch_size = 2,
        split = split,
        base_data_path = "../../../../../../srv/scratch/bic/peter/Report",
        graph_path = "../../../../../../srv/scratch/bic/peter/full-graph",
        vocab_path = "new_vocab_bladderreport.pkl",
        shuffle=True,
        num_workers=0,
        load_in_ram = True
    )
    # for idx,output in enumerate(dataset):
    #     _, _, _, _, labels, _, _= output
    #     print(labels)
    #     class_count[str(labels)] += 1
    #     count += 1

    print(f"length data loader for {split} is {len(loader)}")
    for batch_idx, batch_data in enumerate(loader):
    # Your batch processing code here
        cg, tg, assign_mat, caption_tokens, labels, caption, images, attention_masks= batch_data
        tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
        enc_cap = tokenizer(
            caption, padding=True, truncation=True, max_length=90
        )
        print(caption)
        # print(enc_cap["pad_token_id"])
        print(enc_cap)